In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import express as px

In [5]:
# Reading in the data
events = pd.read_csv("events.csv")
matches = pd.read_csv("ginf.csv")

In [6]:
# Checking structure of the data
print(events.info())
print(matches.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941009 entries, 0 to 941008
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941009 non-null  object 
 1   id_event       941009 non-null  object 
 2   sort_order     941009 non-null  int64  
 3   time           941009 non-null  int64  
 4   text           941009 non-null  object 
 5   event_type     941009 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941009 non-null  int64  
 8   event_team     941009 non-null  object 
 9   opponent       941009 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227459 non-null  float64
 15  shot_outcome   228498 non-null  float64
 16  is_goal        941009 non-null  int64  
 17  location       467067 non-nul

In [7]:
events = pd.merge(events, matches[["id_odsp","league","season"]], on="id_odsp", how="left")

In [9]:
events.head(2)

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,league,season
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,6.0,2.0,0,9.0,2.0,1,1.0,0,D1,2012
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,0,NaN,NaN,0,NaN,0,D1,2012


In [11]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941009 entries, 0 to 941008
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941009 non-null  object 
 1   id_event       941009 non-null  object 
 2   sort_order     941009 non-null  int64  
 3   time           941009 non-null  int64  
 4   text           941009 non-null  object 
 5   event_type     941009 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941009 non-null  int64  
 8   event_team     941009 non-null  object 
 9   opponent       941009 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227459 non-null  float64
 15  shot_outcome   228498 non-null  float64
 16  is_goal        941009 non-null  int64  
 17  location       467067 non-nul

In [21]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10112 entries, 0 to 10111
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_odsp    10112 non-null  object 
 1   link_odsp  10112 non-null  object 
 2   adv_stats  10112 non-null  bool   
 3   date       10112 non-null  object 
 4   league     10112 non-null  object 
 5   season     10112 non-null  int64  
 6   country    10112 non-null  object 
 7   ht         10112 non-null  object 
 8   at         10112 non-null  object 
 9   fthg       10112 non-null  int64  
 10  ftag       10112 non-null  int64  
 11  odd_h      10112 non-null  float64
 12  odd_d      10112 non-null  float64
 13  odd_a      10112 non-null  float64
 14  odd_over   977 non-null    float64
 15  odd_under  977 non-null    float64
 16  odd_bts    977 non-null    float64
 17  odd_bts_n  977 non-null    float64
dtypes: bool(1), float64(7), int64(3), object(7)
memory usage: 1.3+ MB


In [23]:
# Data manipulation
events['is_goal'] = events['is_goal'].astype(bool)
events['bodypart'] = events['bodypart'].astype('category')
events['assist_method'] = events['assist_method'].astype('category')
events['id_odsp'] = events['id_odsp'].astype('category')
events['league'] = events['league'].astype('category')
events['season'] = events['season'].astype('category')
matches['league'] = matches['league'].astype('category')
matches['season'] = matches['season'].astype('category')

In [24]:
# Replacing league levels with more common names
league_mapping = {'D1': 'Bundesliga', 'E0': 'Premier League', 'F1': 'Ligue 1', 'I1': 'Serie A', 'SP1': 'La Liga'}
events['league'] = events['league'].replace(league_mapping)
matches['league'] = matches['league'].replace(league_mapping)

/var/folders/8l/1ycpyghj5yxd26zmz26_lb2r0000gn/T/ipykernel_3069/1577417762.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  events['league'] = events['league'].replace(league_mapping)
/var/folders/8l/1ycpyghj5yxd26zmz26_lb2r0000gn/T/ipykernel_3069/1577417762.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  matches['league'] = matches['league'].replace(league_mapping)


In [ ]:
events.bbbb